In [11]:
path="/data1/yang/data/link_data_rl/frozen_dqn_v7_gamma0_9_False_0.1/model_1200000.ckpt"
meta_path="/data1/yang/data/link_data_rl/frozen_dqn_v7_gamma0_9_False_0.1/model_1200000.ckpt.meta"
import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [7]:
def get_session():
    tf.reset_default_graph()
    tf_config = tf.ConfigProto(
        inter_op_parallelism_threads=1,
        intra_op_parallelism_threads=1)
    tf_config.gpu_options.allow_growth = True
    session = tf.Session(config=tf_config)
    return session

In [12]:
session = get_session()
saver = tf.train.import_meta_graph(meta_path)
saver.restore(session, path)

In [16]:
vs = tf.all_variables()
for x in vs:
    print x.op.name

q_func/Q_tabular
target_q_func/Q_tabular
q_func_1/Q_tabular
target_q_func_1/Q_tabular
beta1_power
beta2_power
q_func/Q_tabular/Adam
q_func/Q_tabular/Adam_1


In [27]:
import numpy as np
reshape=lambda x: np.reshape(x, (8, 5, 4))

rapid = reshape(session.run(vs[0]))
target = reshape(session.run(vs[1]))
print target

[[[  2.49997377e-01   4.99999732e-01   1.24998637e-01   2.49994680e-01]
  [  2.49999791e-01  -2.27246685e-32   6.24948256e-02   1.24996707e-01]
  [  1.24999858e-01   2.18785227e-32   3.12527046e-02   6.24981523e-02]
  [  6.24999292e-02   1.83275894e-32   1.56243853e-02   3.12516540e-02]
  [  3.12499590e-02   7.81412143e-03   1.56272128e-02   1.56219443e-02]]

 [[  4.99997318e-01   9.99999762e-01   2.24636588e-32   2.49997675e-01]
  [  6.80926368e-02  -4.36309688e-02   1.37275551e-03  -5.61551563e-03]
  [  4.59833778e-02   4.40807641e-02   4.64842282e-03   1.13448752e-02]
  [ -2.66335718e-02  -5.94238937e-02  -1.04373647e-02   4.69046924e-03]
  [  1.71907086e-32   3.90779274e-03   7.81346019e-03   1.56249721e-02]]

 [[ -3.14565897e-02  -4.26005386e-02  -5.11881076e-02   5.85232601e-02]
  [ -1.52895655e-02  -1.31422291e-02  -3.75838615e-02   2.33247112e-02]
  [ -4.84169349e-02  -6.38220832e-03  -2.68639959e-02   1.16070751e-02]
  [ -1.53241539e-02  -3.54189649e-02  -8.45466857e-04   1.28

In [34]:
# decode the action meaning: left, down, right, up
mapping = ["L", "D", "R", "U"]
nrow = 8
ncol = 5
optimal_decision=np.zeros(dtype="c", shape=(nrow, ncol))
qvalue = np.zeros(dtype=np.float32, shape=(nrow, ncol))
for i in range(nrow):
    for j in range(ncol):
        vec = target[i, j, :]
        optimal_decision[i,j] = mapping[np.argmax(vec)]
        qvalue[i,j]=np.max(vec)
    
print optimal_decision
np.set_printoptions(precision=2)
print qvalue

[['D' 'L' 'L' 'L' 'L']
 ['D' 'L' 'L' 'U' 'U']
 ['U' 'U' 'U' 'U' 'U']
 ['D' 'U' 'R' 'L' 'U']
 ['R' 'R' 'R' 'R' 'U']
 ['L' 'D' 'U' 'D' 'L']
 ['U' 'U' 'U' 'U' 'L']
 ['U' 'L' 'U' 'R' 'L']]
[[  5.00e-01   2.50e-01   1.25e-01   6.25e-02   3.12e-02]
 [  1.00e+00   6.81e-02   4.60e-02   4.69e-03   1.56e-02]
 [  5.85e-02   2.33e-02   1.16e-02   1.28e-02   7.81e-03]
 [  1.71e-05   1.39e-02   2.19e-02   3.42e-02   3.91e-03]
 [  6.07e-05   2.44e-04   4.88e-04   9.77e-04   1.95e-03]
 [  5.65e-02   5.83e-02   2.44e-04   5.73e-02   2.84e-02]
 [  3.98e-02   5.55e-02   1.22e-04   2.03e-02   1.73e-02]
 [  1.91e-02   7.15e-02   6.10e-05  -2.95e-03   3.38e-02]]


In [24]:
m = [
        "FFFFF",
        "FHHHF",
        "GHHHF",
        "FHHHF",
        "FFFFF",
        "HHFHH",
        "HHFHH",
        "HHSHH",
    ]
m=np.array(m, dtype="c")
print m.shape

(8, 5)


In [35]:
P={0: {0: [(1.0, 0, 0.0, False)], 1: [(1.0, 5, 0.0, False)], 2: [(1.0, 1, 0.0, False)], 3: [(1.0, 0, 0.0, False)]}, 1: {0: [(1.0, 0, 0.0, False)], 1: [(1.0, 6, 0.0, True)], 2: [(1.0, 2, 0.0, False)], 3: [(1.0, 1, 0.0, False)]}, 2: {0: [(1.0, 1, 0.0, False)], 1: [(1.0, 7, 0.0, True)], 2: [(1.0, 3, 0.0, False)], 3: [(1.0, 2, 0.0, False)]}, 3: {0: [(1.0, 2, 0.0, False)], 1: [(1.0, 8, 0.0, True)], 2: [(1.0, 4, 0.0, False)], 3: [(1.0, 3, 0.0, False)]}, 4: {0: [(1.0, 3, 0.0, False)], 1: [(1.0, 9, 0.0, False)], 2: [(1.0, 4, 0.0, False)], 3: [(1.0, 4, 0.0, False)]}, 5: {0: [(1.0, 5, 0.0, False)], 1: [(1.0, 10, 1.0, True)], 2: [(1.0, 6, 0.0, True)], 3: [(1.0, 0, 0.0, False)]}, 6: {0: [(1.0, 6, 0, True)], 1: [(1.0, 6, 0, True)], 2: [(1.0, 6, 0, True)], 3: [(1.0, 6, 0, True)]}, 7: {0: [(1.0, 7, 0, True)], 1: [(1.0, 7, 0, True)], 2: [(1.0, 7, 0, True)], 3: [(1.0, 7, 0, True)]}, 8: {0: [(1.0, 8, 0, True)], 1: [(1.0, 8, 0, True)], 2: [(1.0, 8, 0, True)], 3: [(1.0, 8, 0, True)]}, 9: {0: [(1.0, 8, 0.0, True)], 1: [(1.0, 14, 0.0, False)], 2: [(1.0, 9, 0.0, False)], 3: [(1.0, 4, 0.0, False)]}, 10: {0: [(1.0, 10, 0, True)], 1: [(1.0, 10, 0, True)], 2: [(1.0, 10, 0, True)], 3: [(1.0, 10, 0, True)]}, 11: {0: [(1.0, 11, 0, True)], 1: [(1.0, 11, 0, True)], 2: [(1.0, 11, 0, True)], 3: [(1.0, 11, 0, True)]}, 12: {0: [(1.0, 12, 0, True)], 1: [(1.0, 12, 0, True)], 2: [(1.0, 12, 0, True)], 3: [(1.0, 12, 0, True)]}, 13: {0: [(1.0, 13, 0, True)], 1: [(1.0, 13, 0, True)], 2: [(1.0, 13, 0, True)], 3: [(1.0, 13, 0, True)]}, 14: {0: [(1.0, 13, 0.0, True)], 1: [(1.0, 19, 0.0, False)], 2: [(1.0, 14, 0.0, False)], 3: [(1.0, 9, 0.0, False)]}, 15: {0: [(1.0, 15, 0.0, False)], 1: [(1.0, 20, 0.0, False)], 2: [(1.0, 16, 0.0, True)], 3: [(1.0, 10, 1.0, True)]}, 16: {0: [(1.0, 16, 0, True)], 1: [(1.0, 16, 0, True)], 2: [(1.0, 16, 0, True)], 3: [(1.0, 16, 0, True)]}, 17: {0: [(1.0, 17, 0, True)], 1: [(1.0, 17, 0, True)], 2: [(1.0, 17, 0, True)], 3: [(1.0, 17, 0, True)]}, 18: {0: [(1.0, 18, 0, True)], 1: [(1.0, 18, 0, True)], 2: [(1.0, 18, 0, True)], 3: [(1.0, 18, 0, True)]}, 19: {0: [(1.0, 18, 0.0, True)], 1: [(1.0, 24, 0.0, False)], 2: [(1.0, 19, 0.0, False)], 3: [(1.0, 14, 0.0, False)]}, 20: {0: [(1.0, 20, 0.0, False)], 1: [(1.0, 25, 0.0, True)], 2: [(1.0, 21, 0.0, False)], 3: [(1.0, 15, 0.0, False)]}, 21: {0: [(1.0, 20, 0.0, False)], 1: [(1.0, 26, 0.0, True)], 2: [(1.0, 22, 0.0, False)], 3: [(1.0, 16, 0.0, True)]}, 22: {0: [(1.0, 21, 0.0, False)], 1: [(1.0, 27, 0.0, False)], 2: [(1.0, 23, 0.0, False)], 3: [(1.0, 17, 0.0, True)]}, 23: {0: [(1.0, 22, 0.0, False)], 1: [(1.0, 28, 0.0, True)], 2: [(1.0, 24, 0.0, False)], 3: [(1.0, 18, 0.0, True)]}, 24: {0: [(1.0, 23, 0.0, False)], 1: [(1.0, 29, 0.0, True)], 2: [(1.0, 24, 0.0, False)], 3: [(1.0, 19, 0.0, False)]}, 25: {0: [(1.0, 25, 0, True)], 1: [(1.0, 25, 0, True)], 2: [(1.0, 25, 0, True)], 3: [(1.0, 25, 0, True)]}, 26: {0: [(1.0, 26, 0, True)], 1: [(1.0, 26, 0, True)], 2: [(1.0, 26, 0, True)], 3: [(1.0, 26, 0, True)]}, 27: {0: [(1.0, 26, 0.0, True)], 1: [(1.0, 32, 0.0, False)], 2: [(1.0, 28, 0.0, True)], 3: [(1.0, 22, 0.0, False)]}, 28: {0: [(1.0, 28, 0, True)], 1: [(1.0, 28, 0, True)], 2: [(1.0, 28, 0, True)], 3: [(1.0, 28, 0, True)]}, 29: {0: [(1.0, 29, 0, True)], 1: [(1.0, 29, 0, True)], 2: [(1.0, 29, 0, True)], 3: [(1.0, 29, 0, True)]}, 30: {0: [(1.0, 30, 0, True)], 1: [(1.0, 30, 0, True)], 2: [(1.0, 30, 0, True)], 3: [(1.0, 30, 0, True)]}, 31: {0: [(1.0, 31, 0, True)], 1: [(1.0, 31, 0, True)], 2: [(1.0, 31, 0, True)], 3: [(1.0, 31, 0, True)]}, 32: {0: [(1.0, 31, 0.0, True)], 1: [(1.0, 37, 0.0, False)], 2: [(1.0, 33, 0.0, True)], 3: [(1.0, 27, 0.0, False)]}, 33: {0: [(1.0, 33, 0, True)], 1: [(1.0, 33, 0, True)], 2: [(1.0, 33, 0, True)], 3: [(1.0, 33, 0, True)]}, 34: {0: [(1.0, 34, 0, True)], 1: [(1.0, 34, 0, True)], 2: [(1.0, 34, 0, True)], 3: [(1.0, 34, 0, True)]}, 35: {0: [(1.0, 35, 0, True)], 1: [(1.0, 35, 0, True)], 2: [(1.0, 35, 0, True)], 3: [(1.0, 35, 0, True)]}, 36: {0: [(1.0, 36, 0, True)], 1: [(1.0, 36, 0, True)], 2: [(1.0, 36, 0, True)], 3: [(1.0, 36, 0, True)]}, 37: {0: [(1.0, 36, 0.0, True)], 1: [(1.0, 37, 0.0, False)], 2: [(1.0, 38, 0.0, True)], 3: [(1.0, 32, 0.0, False)]}, 38: {0: [(1.0, 38, 0, True)], 1: [(1.0, 38, 0, True)], 2: [(1.0, 38, 0, True)], 3: [(1.0, 38, 0, True)]}, 39: {0: [(1.0, 39, 0, True)], 1: [(1.0, 39, 0, True)], 2: [(1.0, 39, 0, True)], 3: [(1.0, 39, 0, True)]}}

In [40]:
def state_to_ij(s, nrow=8, ncol=5):
    return (s/ncol, s%col)

def to_s(row, col):
    return row*ncol + col
care = to_s(3, 0)
print(P[care])
care2=to_s(1, 0)
print(P[care2])
# print the intersetion


{0: [(1.0, 15, 0.0, False)], 1: [(1.0, 20, 0.0, False)], 2: [(1.0, 16, 0.0, True)], 3: [(1.0, 10, 1.0, True)]}
{0: [(1.0, 5, 0.0, False)], 1: [(1.0, 10, 1.0, True)], 2: [(1.0, 6, 0.0, True)], 3: [(1.0, 0, 0.0, False)]}
